I want to figure out which machine learning model will best predict counties with high COVID fatality rates. I will train separate models for year one and year two. Hopefully the same model performs well on both years.

First, let's get all the data:

In [1]:
import sys
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import RidgeClassifierCV
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import SVC
import pandas as pd
from sklearn.metrics import roc_auc_score

class config:
    USE_CACHE = True
    CACHE_DIR = "/Users/caseydurfee/msds/data_mining_final_project/cache"

from COVID_data import all_data
data = all_data.get_all_data(config)

Now, we'll get training data for year 1, with a min population of 50,000.

In [9]:
from COVID_data import prepare_model_data

from sklearn.linear_model import LogisticRegressionCV, LassoLarsCV, ElasticNetCV
from sklearn.ensemble import BaggingClassifier, GradientBoostingClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
import pprint

test_models = [ RandomForestClassifier(), 
                SVC(), 
                LogisticRegressionCV(max_iter=10000), 
                RidgeClassifierCV(), 
                AdaBoostClassifier(), 
                BaggingClassifier(),
                GradientBoostingClassifier(),
                LinearSVC(max_iter=10000),
                LassoLarsCV(normalize=False),
                ElasticNetCV()
            ]

roc_auc_scorer = make_scorer(roc_auc_score)

ITERS = 1

for year in [1,2]:
    df, X, y = prepare_model_data.make_train_test(data, year=year, split=False)

    roc_scores = {}
    max_roc = 0.0
    best_model = None
    for throwaway in range(ITERS):
        for model in test_models:
            model_name = repr(model.__class__)

            model = model.fit(X, y) # shouldnt' need this for cross-val

            mean_score = cross_val_score(model, X, y, scoring=roc_auc_scorer).mean()

            if model_name not in roc_scores:
                roc_scores[model_name] = 0.0
            roc_scores[model_name] += mean_score
            if mean_score > max_roc:
                best_model = model
                max_roc = mean_score

    sz = pd.Series(roc_scores)
    print(f">>>> mean scores for year {year}")
    print(sz / ITERS)


dropping ['DEATHS_FIRST_YEAR', 'DEATHS', 'DEATHS_SECOND_YEAR', 'DEATHS_ALPHA', 'DEATHS_DELTA', 'DEATHS_OMICRON', 'DEATH_RATE', 'DEATH_RATE_FIRST_YEAR', 'DEATH_RATE_SECOND_YEAR', 'DEATH_RATE_ALPHA', 'DEATH_RATE_DELTA', 'DEATH_RATE_OMICRON']


/Users/caseydurfee/.virtualenvs/py3-data/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


>>>> mean scores for year 1
<class 'sklearn.ensemble._forest.RandomForestClassifier'>          0.586549
<class 'sklearn.svm._classes.SVC'>                                 0.588568
<class 'sklearn.linear_model._logistic.LogisticRegressionCV'>      0.584277
<class 'sklearn.linear_model._ridge.RidgeClassifierCV'>            0.613578
<class 'sklearn.ensemble._weight_boosting.AdaBoostClassifier'>     0.644440
<class 'sklearn.ensemble._bagging.BaggingClassifier'>              0.603073
<class 'sklearn.ensemble._gb.GradientBoostingClassifier'>          0.612329
<class 'sklearn.svm._classes.LinearSVC'>                           0.636336
<class 'sklearn.linear_model._least_angle.LassoLarsCV'>            0.740688
<class 'sklearn.linear_model._coordinate_descent.ElasticNetCV'>    0.739316
dtype: float64
dropping ['DEATHS_FIRST_YEAR', 'DEATHS', 'DEATHS_SECOND_YEAR', 'DEATHS_ALPHA', 'DEATHS_DELTA', 'DEATHS_OMICRON', 'DEATH_RATE', 'DEATH_RATE_FIRST_YEAR', 'DEATH_RATE_SECOND_YEAR', 'DEATH_RATE_ALPHA',

/Users/caseydurfee/.virtualenvs/py3-data/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/caseydurfee/.virtualenvs/py3-data/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/caseydurfee/.virtualenvs/py3-data/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.011343874434402323, tolerance: 0.01086895973154362
  model = cd_fast.enet_coordinate_descent_gram(
/Users/caseydurfee/.virtualenvs/py3-data/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.012287935580729936, tolerance: 0.0108

>>>> mean scores for year 2
<class 'sklearn.ensemble._forest.RandomForestClassifier'>          0.691530
<class 'sklearn.svm._classes.SVC'>                                 0.680794
<class 'sklearn.linear_model._logistic.LogisticRegressionCV'>      0.714251
<class 'sklearn.linear_model._ridge.RidgeClassifierCV'>            0.693006
<class 'sklearn.ensemble._weight_boosting.AdaBoostClassifier'>     0.696235
<class 'sklearn.ensemble._bagging.BaggingClassifier'>              0.656404
<class 'sklearn.ensemble._gb.GradientBoostingClassifier'>          0.716683
<class 'sklearn.svm._classes.LinearSVC'>                           0.715096
<class 'sklearn.linear_model._least_angle.LassoLarsCV'>            0.865199
<class 'sklearn.linear_model._coordinate_descent.ElasticNetCV'>    0.865058
dtype: float64


Looks like SVC is our big winner.

now, let's use permutation importance to figure out what really matters.

In [13]:
from sklearn.inspection import permutation_importance

df, X, y = prepare_model_data.make_train_test(data, year=1, split=False)

result = permutation_importance(best_model, X, y, 
            n_repeats=100)

perm_importances = pd.Series(result.importances_mean, index=df.columns)

disp = perm_importances.reindex(perm_importances.abs().sort_values().index)

print(disp[disp > 0])

dropping ['DEATHS_FIRST_YEAR', 'DEATHS', 'DEATHS_SECOND_YEAR', 'DEATHS_ALPHA', 'DEATHS_DELTA', 'DEATHS_OMICRON', 'DEATH_RATE', 'DEATH_RATE_FIRST_YEAR', 'DEATH_RATE_SECOND_YEAR', 'DEATH_RATE_ALPHA', 'DEATH_RATE_DELTA', 'DEATH_RATE_OMICRON']
Segregation index         0.000538
Teen Birth Rate           0.000996
Preventable Hosp. Rate    0.001192
% Insufficient Sleep      0.001839
% Alcohol-Impaired        0.002431
EP_MUNIT                  0.004804
% Smokers                 0.005455
% Female                  0.010728
EP_UNINSUR                0.011147
% Asian                   0.012179
% Rural                   0.014280
dtype: float64


In [14]:
df, X, y = prepare_model_data.make_train_test(data, year=2, split=False)

result = permutation_importance(best_model, X, y, 
            n_repeats=100)

perm_importances = pd.Series(result.importances_mean, index=df.columns)

disp = perm_importances.reindex(perm_importances.abs().sort_values().index)

print(disp[disp > 0])

dropping ['DEATHS_FIRST_YEAR', 'DEATHS', 'DEATHS_SECOND_YEAR', 'DEATHS_ALPHA', 'DEATHS_DELTA', 'DEATHS_OMICRON', 'DEATH_RATE', 'DEATH_RATE_FIRST_YEAR', 'DEATH_RATE_SECOND_YEAR', 'DEATH_RATE_ALPHA', 'DEATH_RATE_DELTA', 'DEATH_RATE_OMICRON']
% Excessive Drinking                 0.000042
Length_of_Life_Percentile            0.000042
Years of Potential Life Lost Rate    0.000145
Segregation index                    0.000204
Firearm Fatalities Rate              0.000389
% Insufficient Sleep                 0.000549
% Alcohol-Impaired                   0.000604
EP_MUNIT                             0.000713
% Physically Inactive                0.000755
% Screened                           0.000782
% Smokers                            0.001039
Preventable Hosp. Rate               0.001232
% African American                   0.001436
Physically Unhealthy Days            0.001877
MV Mortality Rate                    0.002030
% Rural                              0.002295
% Asian                 

it seems suspicious that the best model is the only one that tunes itself. Let's try grid search on SVC and see if we can up the success.

In [ ]:
from sklearn.model_selection import cross_val_score, GridSearchCV

parameters = {
    'C': [2**x for x in range(3, 13)],
    'gamma': [2**x for x in range(-15,-5)]
}



grid = GridSearchCV(SVC(), parameters).fit(x_train, y_train)

In [ ]:

scores = [x for x in grid.cv_results_["mean_test_score"]]
scores = np.array(scores).reshape(len(grid.param_grid["C"]), len(grid.param_grid["gamma"]))

print(scores)

In [ ]:
final_model = SVC(C=2**9, gamma=2**-11).fit(x_train, y_train)
